In [1]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.pipeline import make_pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

from bs4 import BeautifulSoup
from langdetect import detect
from urllib.parse import urlsplit

from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Pre Processing

In [2]:
df = pd.read_csv("../Dataset/IMDB Dataset.csv")
df.head()
df = df.rename(columns={'review': 'OriginalReviews'})
df = df.rename(columns={'sentiment': 'OutputSentiment'})
df_subset = df.sample(n=5000, random_state=42).reset_index(drop=True)
df_subset.head()
df_subset['OutputSentiment'].value_counts()

OutputSentiment
positive    2519
negative    2481
Name: count, dtype: int64

In [3]:
df_subset

,OriginalReviews,OutputSentiment
0,I really liked this Summerslam due to the look...,positive
1,Not many television shows appeal to quite as m...,positive
2,The film quickly gets to a major chase scene w...,negative
3,Jane Austen would definitely approve of this o...,positive
4,Expectations were somewhat high for me when I ...,negative
...,...,...
4995,One of eastwood's best movies after he had sep...,positive
4996,My blurred childhood memories have kept the ec...,negative
4997,I love Zombie-Movies and I love amateur-produc...,negative
4998,Chan is in New York and he gets involved with ...,positive


In [4]:
#lowercase
df_subset["OriginalReviews"]=df_subset["OriginalReviews"].apply(lambda x:x.lower())

def remove_punctuation_from_text(text):
    punctuation_to_remove = string.punctuation
    translator = str.maketrans("", "", punctuation_to_remove)
    return text.translate(translator)

# Assuming df_subset is your DataFrame and 'OriginalReviews' is the column to process
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_punctuation_from_text)

# Remove numbers from the 'OriginalReviewss' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('\d+', '')

In [5]:
text = "eastwood's! example, text"
punctuation_to_remove = string.punctuation
translator = str.maketrans("", "", punctuation_to_remove)

text = text.translate(translator)
print(text)


eastwoods example text


In [6]:
# Function to remove stopwords from a text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'OriginalReviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_stopwords)

In [7]:
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Find all matches in the text
    urls = re.findall(url_pattern, text)

    # Remove URLs from the text
    text_without_urls = re.sub(url_pattern, '', text)

    return text_without_urls

# Example usage
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_urls)

In [8]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df_subset["OriginalReviews"] = df_subset["OriginalReviews"].apply(remove_html_tags)

In [9]:
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(clean_text)

In [10]:
def remove_extra_whitespaces(text):
    # Use regular expression to replace multiple whitespaces with a single space
    return re.sub(r'\s+', ' ', text).strip()

df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_extra_whitespaces)

In [11]:
def filter_non_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Create a boolean mask for non-English OriginalReviewss
mask = df_subset['OriginalReviews'].apply(filter_non_english)

# Create a new DataFrame containing only English OriginalReviewss
df_subset = df_subset[mask]

In [12]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech for WordNet lemmatizer
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if the part of speech is not found

# Function to lemmatize a text
def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    return ' '.join(lemmatized_tokens)

# Apply lemmatization to the 'text' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(lemmatize_text)

C:\Users\sdidd\AppData\Local\Temp\ipykernel_20456\486063349.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(lemmatize_text)


In [13]:
df_subset

,OriginalReviews,OutputSentiment
0,really liked summerslam due look arena curtain...,positive
1,many television show appeal quite many differe...,positive
2,film quickly get major chase scene ever increa...,negative
3,jane austen would definitely approve onebr br ...,positive
4,expectation somewhat high go see movie think s...,negative
...,...,...
4995,one eastwoods best movie separate western good...,positive
4996,blur childhood memory keep echo cult serie bel...,negative
4997,love zombiemovies love amateurproductions meat...,negative
4998,chan new york get involve attempt sabotage new...,positive


In [14]:
df_subset.to_csv("../csv/Preprocessed_data.csv",index=False)

## Feature Extraction Using TF-IDF

In [15]:
preprocessed = pd.read_csv('../csv/Preprocessed_data.csv')

In [16]:
num_features_to_keep = 13000

# Create a pipeline with TfidfVectorizer and SelectKBest
pipeline = make_pipeline(TfidfVectorizer(), SelectKBest(f_classif, k=num_features_to_keep))

# Fit and transform your data
X_transformed = pipeline.fit_transform(preprocessed['OriginalReviews'], preprocessed['OutputSentiment'])

# Get the selected feature names
selected_feature_names = pipeline.named_steps['tfidfvectorizer'].get_feature_names_out()[pipeline.named_steps['selectkbest'].get_support()]

# Create a DataFrame with the selected features
selected_features_df = pd.DataFrame(X_transformed.toarray(), columns=selected_feature_names)

# Concatenate the existing DataFrame with the new selected features DataFrame
tfidf_df_13k = pd.concat([preprocessed, selected_features_df], axis=1)

tfidf_df_13k.head()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k.csv")

## CONNOTATIONS

In [ ]:
# Download the VADER lexicon (run this once)
nltk.download('vader_lexicon')
delimiter = '\t'

# Read the text file into a DataFrame
positive = pd.read_csv(r'..\Connotations\positive-words.txt', sep=delimiter, names=['words'])
negative = pd.read_csv(r'..\Connotations\negative-words.txt', sep=delimiter, names=['words'])
connotations = pd.read_csv(r"..\Connotations\connotations.csv")

word_emotion_map = dict(zip(connotations['word'], connotations['emotion']))

def update_counts(review):
    positive_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'positive')
    negative_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'negative')
    return positive_count, negative_count

tfidf_df_13k[['Positive_Connotation_Count', 'Negative_Connotation_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_counts).tolist()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Load positive and negative words from files
positive_words_df = pd.read_csv(r'..\Connotations\positive-words.txt', header=None, names=['words'])
negative_words_df = pd.read_csv(r'..\Connotations\negative-words.txt', header=None, names=['words'])

# Convert DataFrame columns to sets
positive_words = set(positive_words_df['words'].tolist())
negative_words = set(negative_words_df['words'].tolist())

# Assuming 'tfidf_df_13k' is your DataFrame
# Define a function to update counts based on positive and negative words
def update_word_counts(review):
    positive_count = sum(1 for word in review.split() if word in positive_words)
    negative_count = sum(1 for word in review.split() if word in negative_words)
    return positive_count, negative_count

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_Word_Count', 'Negative_Word_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_word_counts).tolist()

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
# Use VADER for sentiment analysis
sid = SentimentIntensityAnalyzer()

def vader_sentiment(review):
    scores = sid.polarity_scores(review)
    return scores['pos'] *100, scores['neg'] * 100

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_VADER_Count', 'Negative_VADER_Count']] = tfidf_df_13k['OriginalReviews'].apply(vader_sentiment).tolist()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k_connotations_vader.csv")

In [ ]:
# text = tfidf_df_13k.iloc[4993]['OriginalReviews']

In [ ]:
#tfidf_df_13k = pd.read_csv("../csv/tfidf_df_13k.csv")

In [ ]:
tfidf_df_13k_connotations = pd.read_csv('../csv/tfidf_df_13k_connotations_vader.csv')

In [ ]:
#tfidf_df_13k_connotations = tfidf_df_13k.drop('Unnamed: 0',axis=1)

In [ ]:
#tfidf_df_13k_connotations = tfidf_df_13k

In [ ]:
tfidf_df_13k_connotations

,Unnamed: 0,OriginalReviews,OutputSentiment,007,007s,0080,010,10000,1010,1010br,...,zucco,zuniga,zunz,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0,really liked summerslam due look arena curtain...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,55,24,7,9,20.3,12.0
1,1,many television show appeal quite many differe...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,77,50,14,6,21.9,2.9
2,2,film quickly get major chase scene ever increa...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20,18,4,4,23.1,9.1
3,3,jane austen would definitely approve onebr br ...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,18,7,5,32.4,10.0
4,4,expectation somewhat high go see movie think s...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,69,42,8,10,15.9,14.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,4992,one eastwoods best movie separate western good...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,6,3,0,36.3,0.0
4993,4993,blur childhood memory keep echo cult serie bel...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29,17,4,4,17.5,13.2
4994,4994,love zombiemovies love amateurproductions meat...,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,28,15,10,2,32.4,4.6
4995,4995,chan new york get involve attempt sabotage new...,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,28,38,8,5,26.6,22.8


In [ ]:
df_statistical = tfidf_df_13k_connotations.drop(columns=['OriginalReviews','Positive_Connotation_Count','Negative_Connotation_Count','Positive_Word_Count','Negative_Word_Count','Positive_VADER_Count','Negative_VADER_Count'], axis=1)
df_statistical.head()

,Unnamed: 0,OutputSentiment,007,007s,0080,010,10000,1010,1010br,102030,...,zoolander,zorina,zorro,zplan,zpm,zu,zucco,zuniga,zunz,zwick
0,0,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
label = LabelEncoder()
df_statistical['OutputSentiment'] = label.fit_transform(df_statistical['OutputSentiment'])

## CHI SQAURE

In [ ]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=5000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_5000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_5k = X[selected_feature_names]
chisq_5k.head()

chisq_5k = pd.concat([chisq_5k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_5k.head()

,Unnamed: 0,007,0080,010,1010,1010br,10br,10yearold,110,110br,...,zombi,zombie,zorro,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,55,24,7,9,20.3,12.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,77,50,14,6,21.9,2.9
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20,18,4,4,23.1,9.1
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,18,7,5,32.4,10.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,69,42,8,10,15.9,14.8


In [ ]:
# This will get the top 8000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=300)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_8000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_8k = X[selected_feature_names]
chisq_8k.head()

chisq_8k = pd.concat([chisq_8k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_8k.head()

,Unnamed: 0,1010,110,210,310,410,710,810,910,act,...,worthless,would,yeah,zombie,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.07521,0.0,0.0,...,0.0,0.000000,0.0,0.0,55,24,7,9,20.3,12.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.109851,0.0,0.0,77,50,14,6,21.9,2.9
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.117799,0.0,0.0,20,18,4,4,23.1,9.1
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.046961,0.0,0.0,22,18,7,5,32.4,10.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,69,42,8,10,15.9,14.8


In [ ]:
# from sklearn.feature_selection import SelectKBest, f_regression

# # For 5000 relevant features
# cor_selector_5k = SelectKBest(f_regression, k=5000)

# # Transform the dataset to reduce dimensions by considering only the relevant features
# X = df_statistical.drop(columns=['OutputSentiment'])
# y = df_statistical['OutputSentiment']
# X_5000 = cor_selector_5k.fit_transform(X, y)

# # Get the indices of the selected features
# selected_feature_indices_5k = cor_selector_5k.get_support(indices=True)

# # Get the names of the selected features
# selected_feature_names_5k = X.columns[selected_feature_indices_5k]

# cor_5k = X[selected_feature_names_5k]
# cor_5k.head()

# cor_5k = pd.concat([cor_5k, tfidf_df_13k_connotations.iloc[:, -6:]], axis=1)
# cor_5k.head()

# # For 8000 relevant features
# cor_selector_8k = SelectKBest(f_regression, k=8000)

# # Transform the dataset to reduce dimensions by considering only the relevant features
# X = df_statistical.drop(columns=['OutputSentiment'])
# y = df_statistical['OutputSentiment']
# X_8000 = cor_selector_8k.fit_transform(X, y)

# # Get the indices of the selected features
# selected_feature_indices_8k = cor_selector_8k.get_support(indices=True)

# # Get the names of the selected features
# selected_feature_names_8k = X.columns[selected_feature_indices_8k]

# cor_8k = X[selected_feature_names_8k]
# cor_8k.head()

# cor_8k = pd.concat([cor_8k, tfidf_df_13k_connotations.iloc[:, -6:]], axis=1)
# cor_8k.head()

In [ ]:
# from sklearn.feature_selection import SelectKBest, mutual_info_regression

# # For 5000 relevant features using mutual information
# info_gain_selector_5k = SelectKBest(mutual_info_regression, k=5000)

# # Transform the dataset to reduce dimensions by considering only the relevant features
# X = df_statistical.drop(columns=['OutputSentiment'])
# y = df_statistical['OutputSentiment']
# X_5000 = info_gain_selector_5k.fit_transform(X, y)

# # Get the indices of the selected features
# selected_feature_indices_5k = info_gain_selector_5k.get_support(indices=True)

# # Get the names of the selected features
# selected_feature_names_5k = X.columns[selected_feature_indices_5k]

# info_gain_5k = X[selected_feature_names_5k]
# info_gain_5k.head()

# info_gain_5k = pd.concat([info_gain_5k, tfidf_df_13k_connotations.iloc[:, -6:]], axis=1)
# info_gain_5k.head()

# # For 8000 relevant features using mutual information
# info_gain_selector_8k = SelectKBest(mutual_info_regression, k=8000)

# # Transform the dataset to reduce dimensions by considering only the relevant features
# X = df_statistical.drop(columns=['OutputSentiment'])
# y = df_statistical['OutputSentiment']
# X_8000 = info_gain_selector_8k.fit_transform(X, y)

# # Get the indices of the selected features
# selected_feature_indices_8k = info_gain_selector_8k.get_support(indices=True)

# # Get the names of the selected features
# selected_feature_names_8k = X.columns[selected_feature_indices_8k]

# info_gain_8k = X[selected_feature_names_8k]
# info_gain_8k.head()

# info_gain_8k = pd.concat([info_gain_8k, tfidf_df_13k_connotations.iloc[:, -6:]], axis=1)
# info_gain_8k.head()

,007,007s,0080,010,1010,1010br,10dirmick,10yearold,1110,112,...,zu,zucco,zunz,zwick,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,55,24,7,9,20.3,12.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,77,50,14,6,21.9,2.9
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20,18,4,4,23.1,9.1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,22,18,7,5,32.4,10.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,69,42,8,10,15.9,14.8


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Assuming chisq_8k is your DataFrame
columns_to_normalize = ['Positive_Connotation_Count', 'Negative_Connotation_Count', 
                         'Positive_Word_Count', 'Negative_Word_Count', 
                         'Positive_VADER_Count', 'Negative_VADER_Count']

scaler = MinMaxScaler()
chisq_8k[columns_to_normalize] = scaler.fit_transform(chisq_8k[columns_to_normalize])

In [ ]:
chisq_8k

,1010,110,210,310,410,710,810,910,act,actually,...,worthless,would,yeah,zombie,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.07521,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.236364,0.121212,0.104478,0.130435,0.269231,0.184332
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.0,0.109851,0.0,0.0,0.336364,0.252525,0.208955,0.086957,0.290451,0.044547
2,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.0,0.117799,0.0,0.0,0.077273,0.090909,0.059701,0.057971,0.306366,0.139785
3,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.0,0.046961,0.0,0.0,0.086364,0.090909,0.104478,0.072464,0.429708,0.153610
4,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.300000,0.212121,0.119403,0.144928,0.210875,0.227343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.0,0.112853,0.0,0.0,0.013636,0.030303,0.044776,0.000000,0.481432,0.000000
4993,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.049861,0.0,...,0.0,0.000000,0.0,0.0,0.118182,0.085859,0.059701,0.057971,0.232095,0.202765
4994,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.113636,0.075758,0.149254,0.028986,0.429708,0.070661
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,...,0.0,0.131418,0.0,0.0,0.113636,0.191919,0.119403,0.072464,0.352785,0.350230


In [ ]:
chisq_8k = chisq_8k.drop(columns=['Unnamed: 0'])

In [ ]:
# Min-max normalization
chisq_8k = (chisq_8k - chisq_8k.min()) / (chisq_8k.max() - chisq_8k.min())

In [ ]:
chisq_8k

,1010,110,210,310,410,710,810,910,act,actually,...,worthless,would,yeah,zombie,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.439033,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.236364,0.121212,0.104478,0.130435,0.269231,0.184332
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.275261,0.0,0.0,0.336364,0.252525,0.208955,0.086957,0.290451,0.044547
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.295174,0.0,0.0,0.077273,0.090909,0.059701,0.057971,0.306366,0.139785
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.117671,0.0,0.0,0.086364,0.090909,0.104478,0.072464,0.429708,0.153610
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.300000,0.212121,0.119403,0.144928,0.210875,0.227343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.282783,0.0,0.0,0.013636,0.030303,0.044776,0.000000,0.481432,0.000000
4993,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.184114,0.0,...,0.0,0.000000,0.0,0.0,0.118182,0.085859,0.059701,0.057971,0.232095,0.202765
4994,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.113636,0.075758,0.149254,0.028986,0.429708,0.070661
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.329301,0.0,0.0,0.113636,0.191919,0.119403,0.072464,0.352785,0.350230


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

numerical_columns = chisq_8k.columns[0:]
# Iterate through each row in the DataFrame
for index, row in chisq_8k.iterrows():
    # Extract the numerical values from the row
    data_row = row[numerical_columns].values
    
    # Generate x-axis values (indices of the elements)
    x_values = np.arange(len(data_row))

    # Create grayscale color values based on data values (normalize to [0, 1])
    color_values = [value / max(data_row) for value in data_row]

    # Create the scatter plot without axes, labels, or color bars
    plt.figure(figsize=(len(data_row) / 100, 0.5))
    plt.scatter(x_values, [0]*len(data_row), c=color_values, cmap='gray', marker='o')
    plt.axis('off')  # Turn off the axes
    plt.margins(0)   # Remove margins
    
    # Save the plot as an image
    plt.savefig(f'row_images/row_{index}_image.png', bbox_inches='tight', pad_inches=0)
    
    # Clear the plot for the next iteration
    plt.close()

# Optionally, show a message indicating that the process is completed
print("Scatter plots saved as images for all rows.")


Scatter plots saved as images for all rows.


In [ ]:
import os
import numpy as np
from PIL import Image

# Create a folder to store the images if it doesn't exist
folder_path = 'row_images'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Function to pad the row with zeros to make it a perfect square
def pad_row(row):
    num_features = len(row)
    side_length = int(np.ceil(np.sqrt(num_features)))  # Calculate the side length of the square grid
    num_padding = side_length * side_length - num_features
    padded_row = np.pad(row.values, (0, num_padding), mode='constant', constant_values=0)
    return padded_row

# Iterate over each row in the data
for index, row in chisq_8k.iterrows():  # Assuming 'chisq_8k' is your DataFrame
    padded_row = pad_row(row)  # Pad the row
    side_length = int(np.sqrt(len(padded_row)))  # Calculate the side length of the square grid
    image_array = padded_row.reshape((side_length, side_length)).astype(np.uint8)  # Reshape into a square grid
    
    # Create PIL Image from image array
    image = Image.fromarray(image_array)
    
    # Save the image in the folder with a descriptive filename
    image_path = os.path.join(folder_path, f'row_{index}_image.png')
    image.save(image_path)
    
    print(f"Image saved for row {index} at path: {image_path}")


Image saved for row 0 at path: row_images\row_0_image.png
Image saved for row 1 at path: row_images\row_1_image.png
Image saved for row 2 at path: row_images\row_2_image.png
Image saved for row 3 at path: row_images\row_3_image.png
Image saved for row 4 at path: row_images\row_4_image.png
Image saved for row 5 at path: row_images\row_5_image.png
Image saved for row 6 at path: row_images\row_6_image.png
Image saved for row 7 at path: row_images\row_7_image.png
Image saved for row 8 at path: row_images\row_8_image.png
Image saved for row 9 at path: row_images\row_9_image.png
Image saved for row 10 at path: row_images\row_10_image.png
Image saved for row 11 at path: row_images\row_11_image.png
Image saved for row 12 at path: row_images\row_12_image.png
Image saved for row 13 at path: row_images\row_13_image.png
Image saved for row 14 at path: row_images\row_14_image.png
Image saved for row 15 at path: row_images\row_15_image.png
Image saved for row 16 at path: row_images\row_16_image.png


## CLASSIFICATION

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# Multinomial Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_scores = cross_val_score(nb_classifier, info_gain_8k, y, cv=5)

print("Multinomial Naive Bayes Cross-Validation Scores:")
print(nb_scores)
print("Mean Accuracy:", np.mean(nb_scores))

# k-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier()
knn_scores = cross_val_score(knn_classifier, info_gain_5k, y, cv=5)

print("\nk-Nearest Neighbors Cross-Validation Scores:")
print(knn_scores)
print("Mean Accuracy:", np.mean(knn_scores))

Multinomial Naive Bayes Cross-Validation Scores:
[0.89       0.888      0.88488488 0.86586587 0.88088088]
Mean Accuracy: 0.8819263263263263

k-Nearest Neighbors Cross-Validation Scores:
[0.701      0.722      0.7047047  0.7047047  0.68668669]
Mean Accuracy: 0.7038192192192192


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Load your data
# Assuming X and y are your features and target variables

# Initialize models
svm_model = SVC(kernel='linear')  # Linear SVM
logistic_model = LogisticRegression()

# Initialize KFold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform 5-fold cross-validation for SVM
svm_scores = cross_val_score(svm_model, info_gain_5k, y, cv=kfold)

# Perform 5-fold cross-validation for Logistic Regression
logistic_scores = cross_val_score(logistic_model, info_gain_8k, y, cv=kfold)

# Display the cross-validation scores
print("SVM Cross-validation scores:", svm_scores)
print("Logistic Regression Cross-validation scores:", logistic_scores)

# Optionally, you can calculate mean and standard deviation of the scores
print("SVM Mean Accuracy:", np.mean(svm_scores))
print("SVM Standard Deviation of Accuracy:", np.std(svm_scores))
print("Logistic Regression Mean Accuracy:", np.mean(logistic_scores))
print("Logistic Regression Standard Deviation of Accuracy:", np.std(logistic_scores))

SVM Cross-validation scores: [0.819      0.837      0.85385385 0.82182182 0.83783784]
Logistic Regression Cross-validation scores: [0.816      0.83       0.84584585 0.83283283 0.82782783]
SVM Mean Accuracy: 0.8339027027027027
SVM Standard Deviation of Accuracy: 0.012579071524074222
Logistic Regression Mean Accuracy: 0.8305013013013014
Logistic Regression Standard Deviation of Accuracy: 0.00957682190918486


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

# Assuming chisq_8k has features and y is the output

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(info_gain_8k, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)

print("Neural Network Accuracy on Test Set:", accuracy)

C:\Users\sdidd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6888 - loss: 0.5629
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9857 - loss: 0.0360
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9984 - loss: 0.0073
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0013
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9998 - loss: 0.0016
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9999 - loss: 7.0379e-04
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 9.3526e-04
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9999 - loss: 3.4759e-04
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 1.6152e-04
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 3.1511e-04
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Neural Network Accuracy on Test Set: 0.914


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Assuming chisq_8k has features and y is the output

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(info_gain_5k, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 
# 
# 
# This is Custom Optimizer
# 
# 
# 

optimizer = tf.keras.optimizers.experimental.Adagrad(
    learning_rate=0.1,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    weight_decay=0.001,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Adagrad',
)


# # Build a simple neural network model
# model = ""
# model = Sequential()
# model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train_scaled, y_train, 
    epochs=50, batch_size=32, 
    validation_split=0.15,  # Using a portion of training set for validation
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)

print("Neural Network Accuracy on Test Set:", accuracy)


AttributeError: module 'keras.optimizers' has no attribute 'experimental'